<a href="https://colab.research.google.com/github/june1st/GoogleColab/blob/master/LipNet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LipNetのテスト

<table class="tfo-notebook-buttons" align="left" >
 <td>
<img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
  </td>
</table>


## 0.最初にやること  
・LipNetのスクリプトはGPUとPython2.7を使います。  
・GoogleColabratory、左上の  「編集→ノートブックの設定」で、  
　　→ ランタイムのタイプ「Python2」  
　　→ ハードウェアアクセラレータ「GPU」  
　に変更してください。

## 1. 最初に現在のGoogleColabランタイムの状況を確認

In [1]:
!ls -la
%pwd

total 20
drwxr-xr-x  1 root root 4096 Feb 11 05:21 .
drwxr-xr-x  1 root root 4096 Feb 11 04:52 ..
drwxr-xr-x  1 root root 4096 Feb  6 17:30 .config
drwxr-xr-x 14 root root 4096 Feb 11 05:53 openpose
drwxr-xr-x  1 root root 4096 Feb  6 17:31 sample_data


u'/content'

In [2]:
!python -c 'import chainer; chainer.print_runtime_info()'

Platform: Linux-4.14.79+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.0.0
NumPy: 1.14.6
CuPy:
  CuPy Version          : 5.2.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 10000
  CUDA Driver Version   : 10000
  CUDA Runtime Version  : 10000
  cuDNN Build Version   : 7301
  cuDNN Version         : 7301
  NCCL Build Version    : 2307
iDeep: 2.0.0.post3


## 2. LipNetのスクリプトをダウンロード

In [3]:
#https://www.tensorflow.org/install/install_sources#common_installation_problems
!git clone https://github.com/rizkiarm/LipNet
%cd LipNet

Cloning into 'LipNet'...
remote: Enumerating objects: 343, done.
remote: Total 343 (delta 0), reused 0 (delta 0), pack-reused 343
Receiving objects: 100% (343/343), 125.84 MiB | 28.22 MiB/s, done.
Resolving deltas: 100% (126/126), done.
Checking out files: 100% (134/134), done.
/content/LipNet


In [13]:
#dlibのバージョンを修正（dlib==19.9.0　→　dlib==19.16.0）
!sed -i s/'dlib==19.4.0'/'dlib==19.16.0'/ setup.py
!sed -i s/'tensorflow-gpu==1.0.1'/'tensorflow-gpu==1.4.0'/ setup.py
!sed -i s/'numpy==1.12.1'/'numpy==1.14.3'/ -i setup.py
!pip install distro
!apt update
!apt install -y cmake
!apt install ffmpeg
!pip install dlib
!pip install boost
!pip install -e .

Ign:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Hit:2 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (157 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
26 packages can be upgraded. Run 'apt list --upgra

CUDA9.0のランタイム準備（LipNetは8.0を使うのでここは不要）

In [0]:
#!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
#!dpkg -i --force-confnew cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
#!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
#!apt-get update
#!apt-get install cuda-9-0

CUDA8.0のランタイム準備（CoocleColaboraroryはCUDA9.2を使用しているのでこれを入れる）  
※ --force-confnewしないと途中で止まるので注意  → http://za.toypark.in/html/2010/06-04.html

In [6]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!dpkg -i --force-confnew cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-8-0

--2019-02-11 06:01:45--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.211.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690 (2.6K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’

cuda-repo-ubuntu160 100%[===================>]   2.63K  --.-KB/s    in 0s      

2019-02-11 06:01:45 (120 MB/s) - ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’ saved [2690/2690]

Selecting previously unselected package cuda-repo-ubuntu1604.
(Reading database ... 120641 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604_8.0.61-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1604 (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604 (8.0.61-1) ...

Configuration f

cuDNN6.0の準備（GoogleColabratoryはcuDNN7.2を使用しているのでこれを入れる）

In [7]:
!wget http://developer.download.nvidia.com/compute/redist/cudnn/v6.0/cudnn-8.0-linux-x64-v6.0.tgz
!tar -xzvf cudnn-8.0-linux-x64-v6.0.tgz
!cp cuda/lib64/libcudnn.so.6.0.21 /usr/lib64-nvidia/libcudnn.so.6
#!cp -P cuda/include/cudnn.h /usr/local/cuda-8.0/include
#!cp -P cuda/lib64/libcudnn* /usr/local/cuda-8.0/lib64/
#!chmod a+r /usr/local/cuda-8.0/lib64/libcudnn*
#!cp -P /usr/local/cuda-8.0/lib64/libcudnn.so.6 /usr/lib64-nvidia
#!cp -P /usr/local/cuda-8.0/lib64/libcudnn.so.6.0.21 /usr/lib64-nvidia

--2019-02-11 06:03:10--  http://developer.download.nvidia.com/compute/redist/cudnn/v6.0/cudnn-8.0-linux-x64-v6.0.tgz
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.211.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201134139 (192M) [application/x-compressed]
Saving to: ‘cudnn-8.0-linux-x64-v6.0.tgz’

cudnn-8.0-linux-x64 100%[===================>] 191.82M  3.31MB/s    in 58s     

2019-02-11 06:04:08 (3.30 MB/s) - ‘cudnn-8.0-linux-x64-v6.0.tgz’ saved [201134139/201134139]

cuda/include/cudnn.h
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.6
cuda/lib64/libcudnn.so.6.0.21
cuda/lib64/libcudnn_static.a


## 3. LipReadingの実行

ダウンロードしたファイルの確認

In [8]:
!ls

assets					 LICENSE	  setup.py
common					 lipnet		  tests
cuda					 lipnet.egg-info  train
cuda-repo-ubuntu1604_8.0.61-1_amd64.deb  predict	  training
cudnn-8.0-linux-x64-v6.0.tgz		 README.md
evaluation				 scripts


モデルファイルの確認

In [9]:
!ls ./evaluation/models

overlapped-weights368.h5  unseen-weights178.h5


使用できるサンプル動画の確認

In [10]:
!ls ./evaluation/samples/GRID

bbaf2n.mpg  lbax4n.mpg	lrwp9a.mpg  pwij3p.mpg	sbwe5n.mpg
brbk7n.mpg  lbbc2a.mpg	lwbsza.mpg  sbia1a.mpg	swiz3n.mpg


LipReadingの実行  
・./evaluation/samples/GRID/***　の部分を他のmpgに帰ると、他のサンプル動画に変えられる  
・最後に出てくる[DECODED]が結果  
　　→ bbaf2n.mpg の結果は 「bin blue at f two now」  
　

In [14]:
!./predict ./evaluation/models/overlapped-weights368.h5 ./evaluation/samples/GRID/bbaf2n.mpg

Using TensorFlow backend.

Loading data from disk...
/usr/local/lib/python2.7/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python2.7/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
Data loaded.

2019-02-11 06:20:08.796358: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-02-11 06:20:08.953146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:892] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA no

## 4. LipReadingの結果に関して
・LipNetの出力は「command(4) + color(4) +preposition(4) + letter(25) + digit(10) + adverb(4)」の順番に限定されている  
　　　→ 認識できるものは以下のモノ　  
　　　　　command　：{bin, lay, place, set}  
　　　　　color　　　 ：{blue, green, red, white}  
　　　　　preposition  ：{at, by, in, with}  
　　　　　letter　　　 ：{A, . . . , Z}\{W}  
　　　　　digit　　　  ：{zero, . . . , nine}  
　　　　　adverb　　 ：{again, now, please, soon}  
 ・詳細は論文「https://arxiv.org/abs/1611.01599」を参照してください。